In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import multiprocessing
from joblib import Parallel, delayed

# Load the dataset from your local machine
file_path = r'C:\Users\Riphah\Desktop\IoT_Intrusion.csv'
print("Loading dataset...")
data = pd.read_csv(file_path)
print("Dataset loaded successfully.")

# Separate features and target variable
X = data.drop(['label'], axis=1)
y = data['label']

# Split the data into training and testing sets
print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data split completed.")

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

# Build the neural network model
def create_model(input_shape, num_classes):
    model = Sequential([
        Dense(128, input_shape=input_shape, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

print("Building the model...")
model = create_model((X_train.shape[1],), num_classes)
print("Model built successfully.")

# Set the number of threads
num_threads = multiprocessing.cpu_count()
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.threading.set_inter_op_parallelism_threads(num_threads)

# Train the model
print("Training the model...")

# Define a function to train the model on a chunk of data
def train_model_chunk(X_chunk, y_chunk):
    return model.train_on_batch(X_chunk, y_chunk)

# Divide the data into chunks based on the number of cores
num_cores = multiprocessing.cpu_count()
X_train_chunks = np.array_split(X_train, num_cores)
y_train_chunks = np.array_split(y_train_categorical, num_cores)

# Train the model on each chunk of data in parallel
results = Parallel(n_jobs=num_cores)(delayed(train_model_chunk)(X_chunk, y_chunk) for X_chunk, y_chunk in zip(X_train_chunks, y_train_chunks))

print("Model trained successfully.")

# Evaluate the model
print("Evaluating the model...")
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Evaluation completed.")
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the trained model
print("Saving the trained model...")
model.save('deepfeedforward_model.h5')
print("Model saved successfully.")
